In [75]:
import pandas as pd

df_train = pd.read_csv("pm_train.csv")
df_test = pd.read_csv("pm_test.csv")

period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()
df_train.label_bc.value_counts()

0    17531
1     3100
Name: label_bc, dtype: int64

In [76]:
df_train.isnull().mean()
df_test.isnull().mean()

Unnamed: 0    0.0
id            0.0
cycle         0.0
setting1      0.0
setting2      0.0
setting3      0.0
s1            0.0
s2            0.0
s3            0.0
s4            0.0
s5            0.0
s6            0.0
s7            0.0
s8            0.0
s9            0.0
s10           0.0
s11           0.0
s12           0.0
s13           0.0
s14           0.0
s15           0.0
s16           0.0
s17           0.0
s18           0.0
s19           0.0
s20           0.0
s21           0.0
ttf           0.0
label_bc      0.0
dtype: float64

In [77]:
df_train.drop( ['Unnamed: 0', 'id','cycle', 'ttf'], axis=1, inplace=True)
df_test.drop( ['Unnamed: 0', 'id', 'cycle', 'ttf'], axis=1, inplace=True)
df_test.head(10)

,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,label_bc
0,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,0
1,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,0
2,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,0
3,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,554.07,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,0
4,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,554.16,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,0
5,0.0012,0.0003,100.0,518.67,642.11,1579.12,1395.13,14.62,21.61,554.22,...,2388.08,8127.46,8.4238,0.03,392,2388,100.0,38.91,23.3467,0
6,-0.0000,0.0002,100.0,518.67,642.11,1583.34,1404.84,14.62,21.61,553.89,...,2388.06,8134.97,8.3914,0.03,391,2388,100.0,38.85,23.3952,0
7,0.0006,-0.0000,100.0,518.67,642.54,1580.89,1400.89,14.62,21.61,553.59,...,2388.06,8125.93,8.4213,0.03,393,2388,100.0,39.05,23.3224,0
8,-0.0036,0.0000,100.0,518.67,641.88,1593.29,1412.28,14.62,21.61,554.49,...,2388.05,8134.15,8.4353,0.03,391,2388,100.0,39.10,23.4521,0
9,-0.0025,-0.0001,100.0,518.67,642.07,1585.25,1398.64,14.62,21.61,554.28,...,2388.06,8134.08,8.4093,0.03,391,2388,100.0,38.87,23.3820,0


In [78]:
x_train = df_train.drop("label_bc", axis=1)
y_train = df_train.label_bc
x_test = df_test.drop("label_bc", axis=1)
y_test = df_test.label_bc

#scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train= pd.DataFrame( scaler.fit_transform(x_train), columns = x_train.columns )
x_test= pd.DataFrame( scaler.fit_transform(x_test), columns = x_test.columns )

In [79]:
print("Before UpSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UpSampling, counts of label '0': {} \n".format(sum(y_train==0)))

Before UpSampling, counts of label '1': 3100
Before UpSampling, counts of label '0': 17531 



In [80]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = 0.4, k_neighbors = 1, random_state=1)
x_train, y_train= sm.fit_resample(x_train, y_train.ravel())

In [81]:
print("Before UpSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UpSampling, counts of label '0': {} \n".format(sum(y_train==0)))

Before UpSampling, counts of label '1': 7012
Before UpSampling, counts of label '0': 17531 



In [82]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical #pandas의 get_dummies
y_train = to_categorical( y_train)
y_test = to_categorical( y_test )

In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add( BatchNormalization(input_shape = (24,)) )
model.add( Dropout(0.25) )
model.add( Dense( 48, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.25) )
model.add( Dense( 24, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.25) )
model.add( Dense( 12, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.25) )
model.add( Dense( 6, activation = "relu"))

model.add( Dense( 2, activation = "softmax"))

model.compile( optimizer = Adam(0.0005),
              loss='categorical_crossentropy',
              metrics = ['acc'])

model.fit( x_train, y_train,
          batch_size=128, epochs=20,
          validation_split=0.3)

Epoch 1/20
135/135 [==============================] - 5s 11ms/step - loss: 0.5030 - acc: 0.8010 - val_loss: 0.3269 - val_acc: 0.9075
Epoch 2/20
135/135 [==============================] - 1s 11ms/step - loss: 0.2655 - acc: 0.9086 - val_loss: 0.2554 - val_acc: 0.9210
Epoch 3/20
135/135 [==============================] - 2s 11ms/step - loss: 0.2011 - acc: 0.9229 - val_loss: 0.2404 - val_acc: 0.9211
Epoch 4/20
135/135 [==============================] - 2s 14ms/step - loss: 0.1748 - acc: 0.9285 - val_loss: 0.2153 - val_acc: 0.9282
Epoch 5/20
135/135 [==============================] - 1s 10ms/step - loss: 0.1598 - acc: 0.9351 - val_loss: 0.2324 - val_acc: 0.9214
Epoch 6/20
135/135 [==============================] - 1s 9ms/step - loss: 0.1530 - acc: 0.9359 - val_loss: 0.2340 - val_acc: 0.9212
Epoch 7/20
135/135 [==============================] - 1s 10ms/step - loss: 0.1449 - acc: 0.9409 - val_loss: 0.2341 - val_acc: 0.9188
Epoch 8/20
135/135 [==============================] - 1s 11ms/step - l

In [84]:
model.evaluate(x_test,y_test)

410/410 [==============================] - 1s 3ms/step - loss: 0.1537 - acc: 0.9385


[0.1537097841501236, 0.9384545087814331]

In [85]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
results = model.predict(x_test)
print('\n', classification_report(np.argmax(y_test, axis = -1), np.argmax(results, axis= -1)))

410/410 [==============================] - 1s 2ms/step

               precision    recall  f1-score   support

           0       1.00      0.94      0.97     12764
           1       0.29      0.97      0.44       332

    accuracy                           0.94     13096
   macro avg       0.64      0.95      0.71     13096
weighted avg       0.98      0.94      0.95     13096



In [86]:
#다른 모형
model = Sequential()

model.add( BatchNormalization(  input_shape = (24,)  ))
model.add( Dropout(0.5)  )
model.add( Dense( 150, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 200, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 180, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 100, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 50, activation = "relu"))


model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 25, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 15, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 7, activation = "relu"))

model.add( BatchNormalization() )
model.add( Dropout(0.5) )
model.add( Dense( 2, activation = "softmax"))

model.compile( optimizer = Adam(0.0005),
              loss='categorical_crossentropy',
              metrics = ['acc'])

model.fit( x_train, y_train,
          batch_size=128, epochs=20,
          validation_split=0.3)

Epoch 1/20
135/135 [==============================] - 12s 22ms/step - loss: 1.2460 - acc: 0.5427 - val_loss: 0.8435 - val_acc: 0.4013
Epoch 2/20
135/135 [==============================] - 2s 17ms/step - loss: 0.9295 - acc: 0.6243 - val_loss: 1.0153 - val_acc: 0.4013
Epoch 3/20
135/135 [==============================] - 2s 15ms/step - loss: 0.7454 - acc: 0.7008 - val_loss: 1.1275 - val_acc: 0.4013
Epoch 4/20
135/135 [==============================] - 2s 15ms/step - loss: 0.6224 - acc: 0.7627 - val_loss: 1.1847 - val_acc: 0.4013
Epoch 5/20
135/135 [==============================] - 2s 16ms/step - loss: 0.5784 - acc: 0.7997 - val_loss: 1.2124 - val_acc: 0.4023
Epoch 6/20
135/135 [==============================] - 2s 15ms/step - loss: 0.5055 - acc: 0.8331 - val_loss: 1.2376 - val_acc: 0.4024
Epoch 7/20
135/135 [==============================] - 3s 20ms/step - loss: 0.4743 - acc: 0.8435 - val_loss: 1.2164 - val_acc: 0.4058
Epoch 8/20
135/135 [==============================] - 2s 15ms/step -

In [87]:
!pip install optuna

In [88]:
def create_model(trial):
    n_hidden = trial.suggest_int('n_hidden', 1, 5)  #1,2,3
    n_units = trial.suggest_int('n_units', 4, 24)  #4~24
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1)

    model = Sequential()
    model.add(Dense(n_units, input_dim=24, activation='relu'))
    for i in range(n_hidden):
        model.add(Dense(n_units, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])
    return model

In [89]:
def objective(trial):
    model = create_model(trial)
    model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=0)
    score = model.evaluate(x_test, y_test, verbose=0)[1]
    return score

In [90]:
import optuna

study = optuna.create_study()
study.optimize(objective, n_trials=10, n_jobs=-1)

[I 2023-08-10 14:57:04,943] A new study created in memory with name: no-name-5864defb-4eed-4998-b481-2721addda521
[I 2023-08-10 14:57:25,076] Trial 1 finished with value: 0.9271533489227295 and parameters: {'n_hidden': 1, 'n_units': 6, 'learning_rate': 0.03582799760220658}. Best is trial 1 with value: 0.9271533489227295.
[I 2023-08-10 14:57:25,860] Trial 0 finished with value: 0.9409743547439575 and parameters: {'n_hidden': 1, 'n_units': 4, 'learning_rate': 0.001448018609884475}. Best is trial 1 with value: 0.9271533489227295.
[I 2023-08-10 14:57:48,562] Trial 2 finished with value: 0.9746487736701965 and parameters: {'n_hidden': 4, 'n_units': 20, 'learning_rate': 0.047495997560491526}. Best is trial 1 with value: 0.9271533489227295.
[I 2023-08-10 14:57:49,486] Trial 3 finished with value: 0.9199755787849426 and parameters: {'n_hidden': 3, 'n_units': 16, 'learning_rate': 0.001174361524702861}. Best is trial 3 with value: 0.9199755787849426.
[I 2023-08-10 14:58:07,315] Trial 4 finished 

In [91]:
print(study.best_params)

{'n_hidden': 4, 'n_units': 22, 'learning_rate': 0.030453972499694965}


In [92]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [93]:
best_model = create_model(study.best_trial)

In [94]:
best_model.evaluate(x_test, y_test)[1]

410/410 [==============================] - 2s 3ms/step - loss: 0.6498 - accuracy: 0.9474


0.9473885297775269

In [95]:
for key, value in study.best_trial.params.items():
    print("{}: {}".format(key, value))

n_hidden: 4
n_units: 22
learning_rate: 0.030453972499694965


In [96]:
from optuna.visualization import plot_contour
plot_contour(study)

In [97]:
from optuna.visualization import plot_param_importances
plot_param_importances(study)